In [ ]:
# Import necessary libraries, if error, !pip3 install the libraries
import os
from dotenv import load_dotenv
import pprint
import requests
from requests_oauthlib import OAuth1

In [ ]:

## Get env variables
# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the .env file, which is located in the `scripts` folder four levels up
env_path = os.path.join(
    current_directory,
    '..', '..', '..', '..',
    'scripts',
    '.env'
)

# Load the environment variables from the .env file
load_dotenv(dotenv_path=env_path)

# NS Credentials
NS_REALM_ID = os.getenv('NS_SANDBOX_REALM_ID')
NS_COMPANY_ID = os.getenv('NS_SANDBOX_COMPANY_ID')
NS_CLIENT_KEY = os.getenv('NS_SANDBOX_CLIENT_KEY')
NS_CLIENT_SECRET = os.getenv('NS_SANDBOX_CLIENT_SECRET')
NS_TOKEN_ID = os.getenv('NS_SANDBOX_TOKEN_ID')
NS_TOKEN_SECRET = os.getenv('NS_SANDBOX_TOKEN_SECRET')

# Sanity check
assert all([
    NS_REALM_ID,
    NS_COMPANY_ID,
    NS_CLIENT_KEY,
    NS_CLIENT_SECRET,
    NS_TOKEN_ID,
    NS_TOKEN_SECRET,
]), "Missing one or more NetSuite env vars."
print("NetSuite credentials loaded from environment variables.")

In [ ]:
# Helper functions

def fetch_all_netsuite_records(auth, endpoint: str):
    """
    Fetch all records from a NetSuite REST API endpoint with pagination support.

    Args:
        endpoint (str): e.g. 'account', 'vendor', 'transaction'

    Returns:
        List of all record items.
    """
    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/record/v1"
    url = f"{base_url}/{endpoint}"

    all_items = []
    offset = 0

    while True:
        paged_url = f"{url}?offset={offset}"
        response = requests.get(paged_url, auth=auth)

        if response.status_code != 200:
            print(f"Request failed: {response.status_code}")
            print(response.text)
            break

        data = response.json()
        items = data.get("items", [])
        all_items.extend(items)

        if not data.get("hasMore", False):
            break

        offset += len(items)

    return all_items

def fetch_record_by_id(auth, endpoint: str, record_id: str):
    """
    Fetch a single NetSuite record by its ID.

    Args:
        endpoint (str): e.g. 'account', 'vendor', 'transaction'
        record_id (str): NetSuite internal ID of the record

    Returns:
        dict: Record data if successful, None otherwise
    """
    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/record/v1"
    url = f"{base_url}/{endpoint}/{record_id}"

    response = requests.get(url, auth=auth)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Request failed with status {response.status_code}")
        print(response.text)
        return None

def create_netsuite_record(auth, endpoint: str, payload: dict):
    """
    Create a new record in NetSuite via REST API.

    Args:
        endpoint (str): e.g. 'account', 'vendor', 'customRecord1'
        payload (dict): The JSON data to create the record

    Returns:
        dict: The created record data if successful, None otherwise
    """
    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/record/v1"
    url = f"{base_url}/{endpoint}"

    response = requests.post(url, json=payload, auth=auth)

    if response.status_code in (200, 201):
        result = response.json()
        record_id = result.get("id")
        print(f"Record created: {endpoint}/{record_id}")
        return result

    elif response.status_code == 204:
        location = response.headers.get("Location")
        if location:
            record_id = location.rstrip("/").split("/")[-1]
            print(f"Record created (204): {endpoint}/{record_id}")
            return {"id": record_id}
        else:
            print("Record created (204), but no Location header provided.")
            return {}

    else:
        print(f"Record creation failed: {response.status_code}")
        print(response.text)
        return None
    
def update_netsuite_record(auth, endpoint: str, record_id: str, payload: dict):
    """
    Update an existing NetSuite record using REST API (PATCH).

    Args:
        auth: OAuth1 auth object
        endpoint (str): e.g. 'customer', 'vendor', 'account'
        record_id (str): Internal ID of the record to update
        payload (dict): Fields and values to update

    Returns:
        dict or None: Updated record data if available, or None
    """
    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/record/v1"
    url = f"{base_url}/{endpoint}/{record_id}"

    response = requests.patch(url, json=payload, auth=auth)

    if response.status_code in (200, 201):
        return response.json()
    elif response.status_code == 204:
        print(f"Record {record_id} updated (204 No Content), but NetSuite returned no response body.")
        return {}
    else:
        print(f"Update failed: {response.status_code}")
        print(response.text)
        return None

def delete_netsuite_record(auth, endpoint: str, record_id: str):    
    """
    Delete a NetSuite record using REST API.

    Args:
        auth: OAuth1 auth object
        endpoint (str): e.g. 'customer', 'vendor', 'account'
        record_id (str): Internal ID of the record to delete

    Returns:
        bool: True if deletion was successful, False otherwise
    """
    base_url = f"https://{NS_COMPANY_ID}.suitetalk.api.netsuite.com/services/rest/record/v1"
    url = f"{base_url}/{endpoint}/{record_id}"

    response = requests.delete(url, auth=auth)

    if response.status_code == 204:
        print(f"Record {record_id} deleted successfully.")
        return True
    else:
        print(f"Deletion failed: {response.status_code}")
        print(response.text)
        return False


In [ ]:
# Authenticate to NetSuite
netsuite_auth = OAuth1(
    client_key=NS_CLIENT_KEY,
    client_secret=NS_CLIENT_SECRET,
    resource_owner_key=NS_TOKEN_ID,
    resource_owner_secret=NS_TOKEN_SECRET,
    signature_method='HMAC-SHA256',
    signature_type='AUTH_HEADER',
    realm=NS_REALM_ID
)

## Fetch Records from an Endpoint

### Step 1: Retrieve Record IDs
Get a list of record IDs for the desired record type.

### Step 2: Fetch Record Details
Retrieve the full details for a specific record using its ID.


In [ ]:
# Fetch all customer ids
customer_ids = fetch_all_netsuite_records(netsuite_auth, endpoint='customer')

# Obtain the first customer ID
customer_id = customer_ids[0]['id']

# With the customer id, get the details of the record
record = fetch_record_by_id(netsuite_auth, 'customer', customer_ids[0]['id'])

# Print the record details
if isinstance(record, dict):
    print(f"Fetched customer record id={record.get('id', customer_id)}")
    print("Fields:", sorted(record.keys()))
else:
    print("Fetched customer record")


## Create a Record via an Endpoint

### Step 1: Specify the Payload
Provide the minimum required fields to create the record (e.g., customers require an associated subsidiary).

### Step 2: Create the Record
Send a `POST` request to the appropriate endpoint with the payload.


In [ ]:
# Prepate the payload for a new customer, subsidiary id is required
payload = {'companyName': "Fake API Company",
               'subsidiary': {
                   'id': '1',}}

# Create a new customer record
created = create_netsuite_record(netsuite_auth, 'customer', payload)

# Print the created record id
if isinstance(created, dict) and created:
    print("Created record id:", created.get("id"))


## Update a Record via an Endpoint

### Step 1: Find the Record ID
Identify the record ID of the record to be updated.  
In this case, use the record created in the previous step.

### Step 2: Prepare the Update Payload
Specify the fields to be updated.  
In this example, we will update the `externalId`.

### Step 3: Update the Record
Send a `PATCH` request to the appropriate endpoint with the update payload.


In [ ]:
# Find the record id of the record to be updated, in this case the one created in the previous step
record_id_to_update = created['id']

# Prepare the payload for the update, in this case we will update the external id
update_payload = {
    "externalId": "7001"
}

# Update the record
result = update_netsuite_record(
    auth=netsuite_auth,
    endpoint="customer",
    record_id=record_id_to_update,
    payload=update_payload
)

# Avoid printing full response payloads in a public notebook
if result is not None:
    print("External ID updated for record id:", record_id_to_update)


## Delete a Record via an Endpoint

### Step 1: Find the Record ID
Identify the record ID of the record to be deleted.  
In this case, use the record created before.

### Step 2: Delete the Record
Send a `DELETE` request to the appropriate endpoint with the record ID.


In [ ]:
# Find the record id of the record to be deleted
record_id_to_delete = created['id']

# Delete the record
delete_netsuite_record(netsuite_auth, 'customer', record_id_to_delete)